In [1]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from qutip import *
import matplotlib.pyplot as plt
from visualisation_tools import *
from model_generator import *
from qsq_protocol import *
from csv_utils import *
from optimizer_utils import *
import datetime
import itertools
import random
plt.rcParams.update({
    "figure.figsize": (10, 6),
    "text.usetex": False, # Remove this line if your code doesn't run/ take too long
    "font.family": 'Times New Roman',
    "figure.dpi": 100,
    "font.size": 14
})
import os
import csv

In [2]:
def average_fidelity_gauge(rho, gate, M, avg_fidelity=None):
    """
    Compute the gauge-corrected average fidelity.
    """
    if avg_fidelity is None:
        avg_fidelity = average_fidelity(rho,gate,M)
    
    eigenvalues, eigenstates = M.eigenstates()
    U_guess = eigenstates[0] @ plus_state.dag() - 1j * eigenstates[1] @ minus_state.dag()
    initial_guess = decompose_U(U_guess)
    bounds = [(-np.pi, np.pi), (0, np.pi/2), (-np.pi, np.pi)]
    result_C = optimize_method('basinhopping', objective_function, initial_guess, rho, gate, M, bounds)
    result_D = optimize_method('differential_evolution', objective_function, initial_guess, rho, gate, M, bounds)
    result = max(1-result_C.fun, 1-result_D.fun, avg_fidelity)
    
    return result


In [3]:
def fill_model_gauge_fidelity(csv_file, flush_every=10):
    # Step 1: read everything
    with open(csv_file, mode='r', newline='') as file:
        reader = csv.DictReader(file)
        fieldnames = reader.fieldnames
        rows = list(reader)

    print(f"{len(rows)} rows to be processed")

    # Step 2: process rows in place
    for i, row in enumerate(rows, start=1):
        if not row.get('model_average_fidelity_gauge'):
            model_data = eval(row['model'])
            rho, gate, M = [Qobj(m) for m in model_data]
            avg_fidelity = float(row['model_average_fidelity'])

            row['model_average_fidelity_gauge'] = average_fidelity_gauge(
                rho, gate, M, avg_fidelity
            )

        # Flush FULL dataset
        if i % flush_every == 0:
            print(f"[{i}/{len(rows)}] flushed ({100*i/len(rows):.1f}%)")
            _rewrite_csv(csv_file, fieldnames, rows)

    # Final flush
    _rewrite_csv(csv_file, fieldnames, rows)


import time
import os
import csv

def _rewrite_csv(csv_file, fieldnames, rows, retries=5, delay=0.2):
    tmp_file = csv_file + ".tmp"

    with open(tmp_file, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)

    for attempt in range(retries):
        try:
            os.replace(tmp_file, csv_file)
            return
        except PermissionError:
            if attempt == retries - 1:
                raise
            time.sleep(delay)



In [6]:
#  Define the CSV file path
csv_file = 'data.csv'
# generate_model_data(csv_file, 'random',10)
generate_model_data(csv_file, 'perturbed',1000)
# Usage
data = load_model_data(csv_file)

# # Access the data by variable name, for example:
# print(data['models'])     # Access the models (now Qobjs)



d:\Program Files\python\Lib\site-packages\qutip\core\data\expm.py:136: LinAlgWarning: Matrix is singular. The result might be inaccurate or the array might not have a square root.
  return Dense(scipy.linalg.sqrtm(matrix.as_ndarray()))


In [7]:
fill_model_gauge_fidelity("data.csv",5)

1030 rows to be processed
[5/1030] flushed (0.5%)
[10/1030] flushed (1.0%)
[15/1030] flushed (1.5%)
[20/1030] flushed (1.9%)
[25/1030] flushed (2.4%)
[30/1030] flushed (2.9%)
[35/1030] flushed (3.4%)
[40/1030] flushed (3.9%)
[45/1030] flushed (4.4%)
[50/1030] flushed (4.9%)
[55/1030] flushed (5.3%)
[60/1030] flushed (5.8%)
[65/1030] flushed (6.3%)
[70/1030] flushed (6.8%)
[75/1030] flushed (7.3%)
[80/1030] flushed (7.8%)
[85/1030] flushed (8.3%)
[90/1030] flushed (8.7%)
[95/1030] flushed (9.2%)
[100/1030] flushed (9.7%)
[105/1030] flushed (10.2%)
[110/1030] flushed (10.7%)
[115/1030] flushed (11.2%)
[120/1030] flushed (11.7%)
[125/1030] flushed (12.1%)
[130/1030] flushed (12.6%)
[135/1030] flushed (13.1%)
[140/1030] flushed (13.6%)
[145/1030] flushed (14.1%)
[150/1030] flushed (14.6%)
[155/1030] flushed (15.0%)
[160/1030] flushed (15.5%)
[165/1030] flushed (16.0%)
[170/1030] flushed (16.5%)
[175/1030] flushed (17.0%)
[180/1030] flushed (17.5%)
[185/1030] flushed (18.0%)
[190/1030] flus

KeyboardInterrupt: 